<a href="https://colab.research.google.com/github/Ictuer/ortools/blob/main/examples/notebook/sat/cp_sat_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# cp_sat_example

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/sat/cp_sat_example.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/sat/samples/cp_sat_example.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [1]:
%pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 25.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatib


Simple solve.

In [3]:
import math
from ortools.sat.python import cp_model
from typing import Dict, List, Any

# Trọng số tối ưu
WEIGHT_STUDENTS_SERVED = 10000
WEIGHT_MINIMIZE_CLASSES = 100
WEIGHT_FEW_SESSIONS = 5
WEIGHT_MISSED_SUBJECTS = 1

# Tham số lớp học
MIN_STUDENTS_PER_CLASS = 3

class ClassSchedulingSolver:
    def __init__(self, sessions: Dict, rooms: Dict, students: Dict):
        self.sessions = sessions
        self.rooms = rooms
        self.students = students
        self.model = cp_model.CpModel()

        # Pre-filter sessions khả thi
        self.feasible_sessions = self._filter_feasible_sessions()

        # Khởi tạo variables và constraints
        self._create_variables()
        self._add_constraints()
        self._set_objective()

    def _filter_feasible_sessions(self) -> Dict:
        """Lọc các session có phòng khả thi"""
        print("=== KIỂM TRA SESSION KHẢ THI ===")
        feasible = {}

        for session_id, session_info in self.sessions.items():
            facility = session_info["facility"]
            slots = session_info["slots"]
            num_students = len(session_info["registers"])

            # Tính số sinh viên tối đa per class
            max_classes = math.floor(num_students / MIN_STUDENTS_PER_CLASS)
            max_students_per_class = math.ceil(num_students / max_classes)

            print(f"\n{session_id}: {num_students} sinh viên, max {max_students_per_class}/lớp -> mở {max_classes} lớp")

            # Tìm phòng khả thi
            has_feasible_room = False
            for room_id, room_info in self.rooms.items():
                if room_info["facility"] != facility:
                    continue

                # Check capacity
                if max_students_per_class > room_info["capacity"]:
                    continue

                # Check time conflict
                conflict = False
                for slot in slots:
                    for busy in room_info["busy"]:
                        if not (slot["end"] <= busy["begin"] or slot["begin"] >= busy["end"]):
                            conflict = True
                            break
                    if conflict:
                        break

                if not conflict:
                    has_feasible_room = True
                    print(f"  ✓ Có thể dùng {room_id}")
                    break

            if has_feasible_room:
                feasible[session_id] = session_info
                print(f"  → Session KHẢ THI")
            else:
                print(f"  → Session KHÔNG KHẢ THI")

        print(f"\nKết quả: {len(feasible)}/{len(self.sessions)} sessions khả thi")
        return feasible

    def _create_variables(self):
        """Tạo variables cho model"""
        self.potential_classes = {}
        self.student_in_class = {}
        self.class_is_opened = {}

        # Tạo potential classes chỉ cho feasible sessions
        for session_id, session_info in self.feasible_sessions.items():
            num_students = len(session_info["registers"])
            max_classes = max(1, math.floor(num_students / MIN_STUDENTS_PER_CLASS))

            for class_idx in range(max_classes):
                class_id = f"{session_id}_C{class_idx}"

                self.potential_classes[class_id] = {
                    "session_id": session_id,
                    "subject": session_info["subject"],
                    "facility": session_info["facility"],
                    "slots": session_info["slots"],
                    "eligible_students": session_info["registers"]
                }

                self.class_is_opened[class_id] = self.model.NewBoolVar(f"opened_{class_id}")

        # Tạo student assignment variables
        for class_id, class_info in self.potential_classes.items():
            for student in class_info["eligible_students"]:
                var_name = f"assign_{student}_{class_id}"
                self.student_in_class[(student, class_id)] = self.model.NewBoolVar(var_name)

        print(f"Tạo {len(self.potential_classes)} lớp tiềm năng")
        print(f"Tạo {len(self.student_in_class)} biến assignment")

    def _add_constraints(self):
        """Thêm constraints"""

        # 1. Mỗi sinh viên tối đa 1 lớp per session
        for session_id in self.feasible_sessions.keys():
            for student in self.students.keys():
                session_classes = []
                for class_id, class_info in self.potential_classes.items():
                    if (class_info["session_id"] == session_id and
                        (student, class_id) in self.student_in_class):
                        session_classes.append(self.student_in_class[(student, class_id)])

                if session_classes:
                    self.model.AddAtMostOne(session_classes)

        # 2. Lớp mở khi đủ sinh viên
        for class_id, class_info in self.potential_classes.items():
            students_in_class = []
            for student in class_info["eligible_students"]:
                if (student, class_id) in self.student_in_class:
                    students_in_class.append(self.student_in_class[(student, class_id)])

            if students_in_class:
                # Mở → đủ min students
                self.model.Add(
                    sum(students_in_class) >= MIN_STUDENTS_PER_CLASS
                ).OnlyEnforceIf(self.class_is_opened[class_id])

                # Không mở → không có student nào
                self.model.Add(
                    sum(students_in_class) == 0
                ).OnlyEnforceIf(self.class_is_opened[class_id].Not())

    def _set_objective(self):
        """Thiết lập objective function"""
        terms = []

        # 1. Maximize students served
        total_served = sum(self.student_in_class.values())
        terms.append(WEIGHT_STUDENTS_SERVED * total_served)

        # 2. Minimize classes opened
        total_classes = sum(self.class_is_opened.values())
        terms.append(-WEIGHT_MINIMIZE_CLASSES * total_classes)

        # 3. Penalty for students with more sessions (to prioritize fewer sessions)
        for student, student_info in self.students.items():
            sessions_count = student_info["registered_sessions"]
            # Phạt sinh viên đăng ký nhiều session
            # sessions_count càng lớn → penalty càng lớn
            penalty = WEIGHT_FEW_SESSIONS * sessions_count

            student_assignments = sum(
                self.student_in_class[(student, class_id)]
                for class_id in self.potential_classes.keys()
                if (student, class_id) in self.student_in_class
            )
            terms.append(-penalty * student_assignments)  # Trừ để penalty

        # 4. Prioritize students with missed subjects
        for student, student_info in self.students.items():
            total_missed = sum(student_info["missed"].values())
            weight = WEIGHT_MISSED_SUBJECTS * total_missed

            student_assignments = sum(
                self.student_in_class[(student, class_id)]
                for class_id in self.potential_classes.keys()
                if (student, class_id) in self.student_in_class
            )
            terms.append(weight * student_assignments)

        self.model.Maximize(sum(terms))

    def solve(self):
        """Giải bài toán"""
        if not self.feasible_sessions:
            return {
                "status": "NO_FEASIBLE_SESSIONS",
                "message": "Không có session nào khả thi"
            }

        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 60

        print(f"\nGiải bài toán với {len(self.potential_classes)} lớp...")
        status = solver.Solve(self.model)

        if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
            return self._extract_solution(solver)
        else:
            return {
                "status": "INFEASIBLE",
                "message": f"Không tìm được solution: {solver.StatusName(status)}"
            }

    def _extract_solution(self, solver):
        """Extract solution từ solver"""
        opened_classes = []

        # Lấy các lớp được mở
        for class_id, class_info in self.potential_classes.items():
            if solver.Value(self.class_is_opened[class_id]):
                students = []
                for student in class_info["eligible_students"]:
                    if ((student, class_id) in self.student_in_class and
                        solver.Value(self.student_in_class[(student, class_id)])):
                        students.append(student)

                opened_classes.append({
                    "class_id": class_id,
                    "session_id": class_info["session_id"],
                    "subject": class_info["subject"],
                    "facility": class_info["facility"],
                    "slots": class_info["slots"],
                    "students": students,
                    "student_count": len(students)
                })

        # Assign rooms
        assigned_classes = self._assign_rooms(opened_classes)

        # Create student assignments
        student_assignments = {}
        for cls in assigned_classes:
            for student in cls["students"]:
                if student not in student_assignments:
                    student_assignments[student] = []
                student_assignments[student].append(cls["class_id"])

        return {
            "status": "OPTIMAL",
            "statistics": {
                "total_students_served": len(student_assignments),
                "total_classes_opened": len(assigned_classes),
                "total_potential_students": len(self.students),
                "feasible_sessions": len(self.feasible_sessions),
                "total_sessions": len(self.sessions)
            },
            "opened_classes": assigned_classes,
            "student_assignments": student_assignments
        }

    def _assign_rooms(self, opened_classes):
        """Assign rooms to classes"""
        print(f"\n=== ASSIGN ROOMS ===")

        # Track room usage
        room_usage = {}
        for room_id, room_info in self.rooms.items():
            room_usage[room_id] = {
                "facility": room_info["facility"],
                "capacity": room_info["capacity"],
                "busy_slots": list(room_info["busy"])
            }

        # Sort classes by student count (descending)
        sorted_classes = sorted(opened_classes, key=lambda x: x["student_count"], reverse=True)

        for cls in sorted_classes:
            best_room = None
            best_capacity = float('inf')

            # Find best room for this class
            for room_id, room_info in room_usage.items():
                if room_info["facility"] != cls["facility"]:
                    continue

                if cls["student_count"] > room_info["capacity"]:
                    continue

                # Check time conflicts
                conflict = False
                for slot in cls["slots"]:
                    for busy in room_info["busy_slots"]:
                        if not (slot["end"] <= busy["begin"] or slot["begin"] >= busy["end"]):
                            conflict = True
                            break
                    if conflict:
                        break

                if not conflict and room_info["capacity"] < best_capacity:
                    best_room = room_id
                    best_capacity = room_info["capacity"]

            if best_room:
                cls["room"] = best_room
                # Update room busy slots
                for slot in cls["slots"]:
                    room_usage[best_room]["busy_slots"].append(slot)
                print(f"  {cls['class_id']} → {best_room}")
            else:
                cls["room"] = None
                print(f"  {cls['class_id']} → NO ROOM (should not happen)")

        return sorted_classes

def main():
    """Test function"""

    sessions = {
        "SS1": {
            "subject": "Math",
            "facility": "NEU",
            "slots": [{"begin": 8, "end": 10}, {"begin": 14, "end": 16}],
            "registers": ["alice", "bob", "charlie", "david", "eva", "frank", "grace"]
        },
        "SS2": {
            "subject": "Physics",
            "facility": "NEU",
            "slots": [{"begin": 10, "end": 12}],
            "registers": ["alice", "bob", "charlie", "david", "eva"]
        },
        "SS3": {
            "subject": "English",
            "facility": "FTU",
            "slots": [{"begin": 9, "end": 11}, {"begin": 13, "end": 15}],
            "registers": ["alice", "charlie", "henry", "iris", "jack", "kate"]
        },
        "SS4": {
            "subject": "Chemistry",
            "facility": "FTU",
            "slots": [{"begin": 15, "end": 17}],
            "registers": ["bob", "david", "henry", "iris"]
        }
    }

    rooms = {
        "NEU_R1": {
            "busy": [{"begin": 12, "end": 13}],
            "facility": "NEU",
            "capacity": 4
        },
        "NEU_R2": {
            "busy": [{"begin": 7, "end": 8}, {"begin": 16, "end": 18}],
            "facility": "NEU",
            "capacity": 6
        },
        "FTU_R1": {
            "busy": [{"begin": 11, "end": 13}],
            "facility": "FTU",
            "capacity": 3
        },
        "FTU_R2": {
            "busy": [{"begin": 17, "end": 19}],
            "facility": "FTU",
            "capacity": 8
        }
    }

    students = {
        "alice": {"missed": {"Math": 3, "English": 1}, "registered_sessions": 3},
        "bob": {"missed": {"Physics": 2}, "registered_sessions": 3},
        "charlie": {"missed": {"Math": 1, "English": 2}, "registered_sessions": 3},
        "david": {"missed": {"Chemistry": 1}, "registered_sessions": 3},
        "eva": {"missed": {}, "registered_sessions": 2},
        "frank": {"missed": {"Math": 4}, "registered_sessions": 1},
        "grace": {"missed": {"Math": 2}, "registered_sessions": 1},
        "henry": {"missed": {"English": 3, "Chemistry": 2}, "registered_sessions": 2},
        "iris": {"missed": {"English": 1, "Chemistry": 3}, "registered_sessions": 2},
        "jack": {"missed": {"English": 2}, "registered_sessions": 1},
        "kate": {"missed": {}, "registered_sessions": 1}
    }

    # Solve
    solver = ClassSchedulingSolver(sessions, rooms, students)
    result = solver.solve()

    # Print results
    print("\n" + "="*80)
    print("KẾT QUẢ")
    print("="*80)

    if result["status"] == "OPTIMAL":
        stats = result["statistics"]
        print(f"Thống kê:")
        print(f"  - Sessions khả thi: {stats['feasible_sessions']}/{stats['total_sessions']}")
        print(f"  - Sinh viên được phục vụ: {stats['total_students_served']}/{stats['total_potential_students']}")
        print(f"  - Lớp được mở: {stats['total_classes_opened']}")

        print(f"\nCác lớp được mở:")
        for cls in result["opened_classes"]:
            room_info = f" → {cls['room']}" if cls['room'] else " → NO ROOM"
            print(f"  {cls['class_id']}: {cls['subject']} ({cls['student_count']} SV){room_info}")
            print(f"    Students: {', '.join(cls['students'])}")
            print(f"    Slots: {cls['slots']}")
            print()

    else:
        print(f"Lỗi: {result['message']}")

if __name__ == "__main__":
    main()

=== KIỂM TRA SESSION KHẢ THI ===

SS1: 7 sinh viên, max 4/lớp -> mở 2 lớp
  ✓ Có thể dùng NEU_R1
  → Session KHẢ THI

SS2: 5 sinh viên, max 5/lớp -> mở 1 lớp
  ✓ Có thể dùng NEU_R2
  → Session KHẢ THI

SS3: 6 sinh viên, max 3/lớp -> mở 2 lớp
  ✓ Có thể dùng FTU_R1
  → Session KHẢ THI

SS4: 4 sinh viên, max 4/lớp -> mở 1 lớp
  ✓ Có thể dùng FTU_R2
  → Session KHẢ THI

Kết quả: 4/4 sessions khả thi
Tạo 6 lớp tiềm năng
Tạo 35 biến assignment

Giải bài toán với 6 lớp...

=== ASSIGN ROOMS ===
  SS1_C1 → NO ROOM (should not happen)
  SS3_C1 → FTU_R2
  SS2_C0 → NEU_R2
  SS4_C0 → FTU_R2

KẾT QUẢ
Thống kê:
  - Sessions khả thi: 4/4
  - Sinh viên được phục vụ: 11/11
  - Lớp được mở: 4

Các lớp được mở:
  SS1_C1: Math (7 SV) → NO ROOM
    Students: alice, bob, charlie, david, eva, frank, grace
    Slots: [{'begin': 8, 'end': 10}, {'begin': 14, 'end': 16}]

  SS3_C1: English (6 SV) → FTU_R2
    Students: alice, charlie, henry, iris, jack, kate
    Slots: [{'begin': 9, 'end': 11}, {'begin': 13, 'en